In [1]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
n_folds = 5
n_runs = 10

# Simulate results with hierarchical structure
# Each fold has a slightly different mean (different data partitions)
# Each run within a fold has some noise (run-to-run variability)

fold_base_means = np.array([0.85, 0.87, 0.84, 0.86, 0.88])
within_fold_std = 0.02

# Generate data: shape (n_folds, n_runs)
results = np.zeros((n_folds, n_runs))
for fold in range(n_folds):
    results[fold, :] = np.random.normal(
        loc=fold_base_means[fold],
        scale=within_fold_std,
        size=n_runs
    )

print("=" * 60)
print("VARIANCE DECOMPOSITION SIMULATION")
print("=" * 60)
print(f"\nExperimental Design: {n_folds} folds × {n_runs} runs = {n_folds * n_runs} total runs\n")

# Display results by fold
print("Results by Fold:")
print("-" * 60)
for fold in range(n_folds):
    fold_mean = results[fold, :].mean()
    fold_std = results[fold, :].std(ddof=0)
    print(f"Fold {fold + 1}: Mean = {fold_mean:.4f}, Std = {fold_std:.4f}")

print("\n" + "=" * 60)
print("METHOD 1: NAIVE POOLING (all 50 values)")
print("=" * 60)

# Method 1: Naive approach - treat all 50 runs as independent
all_values = results.flatten()
naive_mean = all_values.mean()
naive_std = all_values.std(ddof=0)

print(f"Mean: {naive_mean:.4f}")
print(f"Std:  {naive_std:.4f}")

print("\n" + "=" * 60)
print("METHOD 2: LAW OF TOTAL VARIANCE")
print("=" * 60)

# Method 2: Law of Total Variance
# Step 1: Calculate fold-level statistics
fold_means = results.mean(axis=1)  # Mean of each fold
fold_vars = results.var(axis=1, ddof=0)  # Variance within each fold

print("\nFold-level statistics:")
for fold in range(n_folds):
    print(f"Fold {fold + 1}: Mean = {fold_means[fold]:.4f}, Var = {fold_vars[fold]:.6f}")

# Step 2: Between-fold variance
overall_mean = fold_means.mean()
var_between = fold_means.var(ddof=0)

# Step 3: Within-fold variance (average)
var_within = fold_vars.mean()

# Step 4: Total variance
total_var = var_between + var_within
total_std = np.sqrt(total_var)

print(f"\nOverall Mean: {overall_mean:.4f}")
print(f"\nVariance Components:")
print(f"  Between-fold variance: {var_between:.6f} (std: {np.sqrt(var_between):.4f})")
print(f"  Within-fold variance:  {var_within:.6f} (std: {np.sqrt(var_within):.4f})")
print(f"  Total variance:        {total_var:.6f}")
print(f"\nTotal Std: {total_std:.4f}")

print("\n" + "=" * 60)
print("COMPARISON")
print("=" * 60)
print(f"Naive pooling std:     {naive_std:.4f}")
print(f"Total variance std:    {total_std:.4f}")
print(f"Difference:            {total_std - naive_std:.4f}")
print(f"Relative difference:   {((total_std - naive_std) / naive_std * 100):.2f}%")

print("\n" + "=" * 60)
print("INTERPRETATION")
print("=" * 60)
print("The total variance method is LARGER because it properly accounts")
print("for both sources of variability:")
print("  1. Between-fold variance (different data partitions)")
print("  2. Within-fold variance (run-to-run noise)")
print("\nThe naive method underestimates uncertainty by treating")
print("non-independent runs (same fold) as independent samples.")
print("=" * 60)

VARIANCE DECOMPOSITION SIMULATION

Experimental Design: 5 folds × 10 runs = 50 total runs

Results by Fold:
------------------------------------------------------------
Fold 1: Mean = 0.8590, Std = 0.0137
Fold 2: Mean = 0.8542, Std = 0.0143
Fold 3: Mean = 0.8356, Std = 0.0154
Fold 4: Mean = 0.8538, Std = 0.0217
Fold 5: Mean = 0.8749, Std = 0.0171

METHOD 1: NAIVE POOLING (all 50 values)
Mean: 0.8555
Std:  0.0209

METHOD 2: LAW OF TOTAL VARIANCE

Fold-level statistics:
Fold 1: Mean = 0.8590, Var = 0.000188
Fold 2: Mean = 0.8542, Var = 0.000205
Fold 3: Mean = 0.8356, Var = 0.000238
Fold 4: Mean = 0.8538, Var = 0.000472
Fold 5: Mean = 0.8749, Var = 0.000292

Overall Mean: 0.8555

Variance Components:
  Between-fold variance: 0.000158 (std: 0.0126)
  Within-fold variance:  0.000279 (std: 0.0167)
  Total variance:        0.000438

Total Std: 0.0209

COMPARISON
Naive pooling std:     0.0209
Total variance std:    0.0209
Difference:            0.0000
Relative difference:   0.00%

INTERPRETATI